In [19]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
# Add the parent directory of 'src' to the Python path
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Longman phrase and anki deck generation
A more memorable way to learn the core vocabulary as defined in Longman Communications vocab lists, we take the vocab and use an LLM
to generate phrases using it.

## Longman 1000, 2000 and 3000 already provided
Enlish phrases for the longman vocab have already been created and can be found in the 'data' folder

In [32]:
from src.utils import get_longman_verb_vocab_dict
from src.phrase import generate_phrases_from_vocab_dict

file_path = '../data/longman-communication-3000.json'
vocab_dict = get_longman_verb_vocab_dict(file_path, "S3") #S1 = 1st 1000 words used in Speech, options are S1-3 and W1-3

In [33]:
#uses LLM calls - it sometimes generates phrases terminated too early (e.g. Mind the pot on the), so advise you scan through and check

#english phrases only initially
longman_phrases = generate_phrases_from_vocab_dict(vocab_dict, max_iterations=15)

Iteration 1/15
Generated 101 phrases
We have 127 verbs and 728 vocab words left
Iteration 2/15
Generated 106 phrases
We have 58 verbs and 660 vocab words left
Iteration 3/15
Generated 105 phrases
We have 3 verbs and 594 vocab words left
Iteration 4/15
Generated 98 phrases
We have 195 verbs and 520 vocab words left
Iteration 5/15
Generated 105 phrases
We have 112 verbs and 434 vocab words left
Iteration 6/15
Generated 103 phrases
We have 43 verbs and 344 vocab words left
Iteration 7/15
Generated 87 phrases
We have 2 verbs and 307 vocab words left
Iteration 8/15
Generated 99 phrases
We have 250 verbs and 272 vocab words left
Iteration 9/15
Generated 101 phrases
We have 173 verbs and 232 vocab words left
Iteration 10/15
Generated 109 phrases
We have 95 verbs and 161 vocab words left
Iteration 11/15
Generated 100 phrases
We have 37 verbs and 136 vocab words left
Iteration 12/15
Generated 60 phrases - with minimal phrase prompt
We have 30 verbs and 25 vocab words left
Iteration 13/15
Genera

In [34]:
with open("../outputs/longman/longman_3000_phrases.txt", "w", encoding="utf-8") as f:
    for phrase in longman_phrases:
        f.write(phrase + "\n")

## Create an Anki deck from thoses phrases

Or, load one of the english Longman phrase lists in data/ already created and apply to your language

This function:
1. translates
2. generates audio using text to speech
3. packages up the text and audio into several anki decks (in batches), that can be imported into Anki.

The deck_name will is used to derive the deck_id and so despite there being several *.apkg files created, these will all merge successfully into the same deck


In [37]:

from src.anki import create_anki_deck_from_english_phrase_list

_ = await create_anki_deck_from_english_phrase_list(longman_phrases[:3], deck_name="Longman 3000 - Swedish", anki_filename_prefix="longman_3000_swedish", batch_size=50)

Beginning translation for anki
Translated phrases

Text-to-speech for phrases done

Anki deck exported to ../outputs/longman\longman_3000_0_anki_deck.apkg
Cleanup of temporary MP3 files completed.
